In [76]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

In [77]:
#pts = 30000
mdat = pd.read_csv('cleanCMURevenueAdjusted.tsv', delimiter='\t',encoding='utf-8-sig')
plotsum = mdat[mdat.columns[7]]
mdat["Genres"] = mdat.apply(lambda row: eval(row["Genres"]), axis=1)
#g3 = mdat["Genres"]
gen = mdat["Genres"]
titles = mdat["Name"]
mdat.head()

#Split
Train2, Test2 = train_test_split(mdat, test_size=0.3)
#Get plot data
TrainPlot = Train2[mdat.columns[7]]
TestPlot = Test2[mdat.columns[7]]
#Get genre data
TrainGen = Train2["Genres"]
TestGen = Test2["Genres"]
AllGen = gen

In [78]:
#Tf-idf the training plots
TrainVec = TfidfVectorizer(max_features = 3000, stop_words = stopwords.words('english'))
TrainPlotsVec = TrainVec.fit_transform(TrainPlot)
#fit the test plots to this tf-idf
TestPlotsVec =  TrainVec.transform(TestPlot)

In [79]:
#Find the points with genre lists that include comedy
comAll = np.zeros((len(AllGen),))
for jj in range(len(AllGen)):
    if any(x == 'Comedy' for x in AllGen.iloc[jj]):
        comAll[jj] = 1

#Find the points with genre lists that include comedy
comTest = np.zeros((len(TestGen),))
for jj in range(len(TestGen)):
    if any(x == 'Comedy' for x in TestGen.iloc[jj]):
        comTest[jj] = 1
        
comTrain = np.zeros((len(TrainGen),))
for jj in range(len(TrainGen)):
    if any(x == 'Comedy' for x in TrainGen.iloc[jj]):
        comTrain[jj] = 1

In [80]:
#Find the points with genre lists that include horror or thriller
horTrain = np.zeros((len(TrainGen),))
for jj in range(len(TrainGen)):
    if any(x == 'Horror' or x == 'Thriller' for x in TrainGen.iloc[jj]):
        horTrain[jj] = 1
        
horTest = np.zeros((len(TestGen),))
for jj in range(len(TestGen)):
    if any(x == 'Horror' or x == 'Thriller' for x in TestGen.iloc[jj]):
        horTest[jj] = 1

In [81]:
#GBC classify comedy
abc = GradientBoostingClassifier()#n_estimators = 5, learning_rate = 1.0, max_depth = 1, random_state = 0 )
m = abc.fit(TrainPlotsVec , comTrain)
score = m.score(TestPlotsVec, comTest)
print(score)

0.765676828305165


(N, learnrate, acc)
(600, 1, 0.7691) 
(400, 1, 0.7752)
(200, 1, 0.7728)
(400, 1.2, 0.7613)
(400, 0.8, 0.7783)
(400, 0.6, 0.78107)
(600, 0.4, 0.78328)
(800, 0.3, 0.7835)
(1000, 0.3, 0.7821)
(1000, 0.4, 0.78344)
(800, 0.4, 0.78249)

In [82]:
#Adaboost classify comedy
AB_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),algorithm="SAMME.R",n_estimators=800, learning_rate = 0.3)
ABCL = AB_clf.fit(TrainPlotsVec , comTrain)
score = ABCL.score(TestPlotsVec, comTest)

#abc = GradientBoostingClassifier()#n_estimators = 5, learning_rate = 1.0, max_depth = 1, random_state = 0 )
#m = abc.fit(TrainPlotsVec , comTrain)
#score = m.score(TestPlotsVec, comTest)
print(score)

0.7828147212130785


In [83]:
#% of points that are not comedy
1-sum(comTest)/len(comTest) 

0.7475122413520771

In [84]:
#GBC classify horror
abc = GradientBoostingClassifier()#n_estimators = 5, learning_rate = 1.0, max_depth = 1, random_state = 0 )
m = abc.fit(TrainPlotsVec , horTrain)
horscore = m.score(TestPlotsVec, horTest)
print(horscore)

0.8429158110882957


In [85]:
AB_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),algorithm="SAMME.R",n_estimators=800, learning_rate = 0.3)
ABCL = AB_clf.fit(TrainPlotsVec , horTrain)
score = ABCL.score(TestPlotsVec, horTest)
print(score)

0.8571315747907123


In [86]:
BB_clf = BernoulliNB(alpha=0.00001)
BBCL = BB_clf.fit(TrainPlotsVec , horTrain)
score = BBCL.score(TestPlotsVec, horTest)
print(score)

0.7212130785026062


In [87]:
BB_clf = MultinomialNB(alpha = 0.000001)
BBCL = BB_clf.fit(TrainPlotsVec , horTrain)
score = BBCL.score(TestPlotsVec, horTest)
print(score)

0.8386510819775707


(N, learnrate, acc)
(400, 1, 0.8428) 
(500, 1, 0.84307)
(600, 1, 0.8425)
(500, 0.8, 0.849549)
(500, 0.6, 0.85373)
(500, 0.4, 0.85318)
(600, 0.4, 0.85405)
(800, 0.2, 0.850)
(1000, 0.3, 0.8533)
(1000, 0.4, 0.85547)
(1200, 0.4, 0.85642)
(1400, 0.4, 0.85499)

In [88]:
1-sum(horTest)/len(horTest) 

0.7863686621386827